In [4]:
###Introduction to Data Ingestion
import os
from typing import List,Dict,Any
import pandas as pd



In [6]:
from langchain_core.documents import Document
from langchain.text_splitter import (RecursiveCharacterTextSplitter, TokenTextSplitter,CharacterTextSplitter)
print("Set up completed")

Set up completed


Understanding Document Structure In LangChain   


In [7]:
doc=Document(
    page_content="This will be the content that will be embedded and splitted",
    metadata={
        "source": "test.txt",
        "page":1,
        "author":"Ragu",
        "date":"2025-08-18",
        "custom":"custom metadata"},
)
print("doc structure")
print(doc)
print("\n")
print("doc metadata")
print(doc.metadata)
print("\n")
print("doc page_content")
print(doc.page_content)

doc structure
page_content='This will be the content that will be embedded and splitted' metadata={'source': 'test.txt', 'page': 1, 'author': 'Ragu', 'date': '2025-08-18', 'custom': 'custom metadata'}


doc metadata
{'source': 'test.txt', 'page': 1, 'author': 'Ragu', 'date': '2025-08-18', 'custom': 'custom metadata'}


doc page_content
This will be the content that will be embedded and splitted


In [8]:
type(doc)

langchain_core.documents.base.Document

text files


In [ ]:
import os
os.makedirs("data",exist_ok=True)

In [26]:
sample_texts={
    "data/text_files/python.txt":"""Python is an interpreted, high-level and general-purpose programming language. Python's design philosophy emphasizes code readability with its notable use of significant whitespace. Its language constructs and object-oriented approach aim to help programmers write clear, logical code for small and large-scale projects.AI LLM Security Session Summary
This session covers real-world security risks in Large Language Model (LLM) deployments, drawn from penetration testing and recent research. Attendees will learn about key vulnerabilities, see a live demo of a novel prompt extraction attack (revealing system prompts without model access), and understand why traditional security tools fall short. The talk also covers how attackers exploit generative AI and provides practical defenses to strengthen AI LLM security posture.

Why Attend:

Understand common vulnerabilities in LLM applications.

Learn from Black Hat trainers and pen testers real-world findings.

See live demonstrations of prompt-based attacks.

Gain actionable strategies to defend against AI-specific threats.""",
   
}
for filepath,content in sample_texts.items():
    with open(filepath,"w",encoding="utf-8") as f:
        f.write(content)
print("sample file created")

sample file created


TextLoader-Read Single File

In [27]:
from langchain.document_loaders import TextLoader
from langchain_community.document_loaders import TextLoader

loader=TextLoader("data/text_files/python.txt",encoding="utf-8")
doc=loader.load()
print(type(doc))
print(doc)


<class 'list'>
[Document(metadata={'source': 'data/text_files/python.txt'}, page_content="Python is an interpreted, high-level and general-purpose programming language. Python's design philosophy emphasizes code readability with its notable use of significant whitespace. Its language constructs and object-oriented approach aim to help programmers write clear, logical code for small and large-scale projects.AI LLM Security Session Summary\nThis session covers real-world security risks in Large Language Model (LLM) deployments, drawn from penetration testing and recent research. Attendees will learn about key vulnerabilities, see a live demo of a novel prompt extraction attack (revealing system prompts without model access), and understand why traditional security tools fall short. The talk also covers how attackers exploit generative AI and provides practical defenses to strengthen AI LLM security posture.\n\nWhy Attend:\n\nUnderstand common vulnerabilities in LLM applications.\n\nLearn

DirectoryLoader=Multiple text files

In [30]:
from langchain.document_loaders import DirectoryLoader
dir_loader=DirectoryLoader(
    "data/text_files",
glob="**/*.txt",
loader_cls=TextLoader,
loader_kwargs={"encoding":"utf-8"},
show_progress=True
)
documents=dir_loader.load()
for i,doc in enumerate(documents):
    print(f"Document {i+1}: {doc.metadata['source']}")
    

100%|██████████| 1/1 [00:00<00:00, 1474.79it/s]

Document 1: data/text_files/python.txt


Text splitter

In [31]:
from langchain.text_splitter import (RecursiveCharacterTextSplitter, TokenTextSplitter,CharacterTextSplitter)
print(documents)

[Document(metadata={'source': 'data/text_files/python.txt'}, page_content="Python is an interpreted, high-level and general-purpose programming language. Python's design philosophy emphasizes code readability with its notable use of significant whitespace. Its language constructs and object-oriented approach aim to help programmers write clear, logical code for small and large-scale projects.AI LLM Security Session Summary\nThis session covers real-world security risks in Large Language Model (LLM) deployments, drawn from penetration testing and recent research. Attendees will learn about key vulnerabilities, see a live demo of a novel prompt extraction attack (revealing system prompts without model access), and understand why traditional security tools fall short. The talk also covers how attackers exploit generative AI and provides practical defenses to strengthen AI LLM security posture.\n\nWhy Attend:\n\nUnderstand common vulnerabilities in LLM applications.\n\nLearn from Black Hat

In [38]:
###Method 1: character text splitter
text=documents[0].page_content
text
character_text_splitter=CharacterTextSplitter(chunk_size=1000,chunk_overlap=200,length_function=len)
char_chunks=character_text_splitter.split_text(text)
print(f"created {len(char_chunks)} chunks")
print(f"First chunk: {char_chunks[0][:10]}...")
print("\n__________________")
print(char_chunks[0])
print("\n__________________")
print(char_chunks[1])
print("\n__________________")


created 2 chunks
First chunk: Python is ...

__________________
Python is an interpreted, high-level and general-purpose programming language. Python's design philosophy emphasizes code readability with its notable use of significant whitespace. Its language constructs and object-oriented approach aim to help programmers write clear, logical code for small and large-scale projects.AI LLM Security Session Summary
This session covers real-world security risks in Large Language Model (LLM) deployments, drawn from penetration testing and recent research. Attendees will learn about key vulnerabilities, see a live demo of a novel prompt extraction attack (revealing system prompts without model access), and understand why traditional security tools fall short. The talk also covers how attackers exploit generative AI and provides practical defenses to strengthen AI LLM security posture.

Why Attend:

Understand common vulnerabilities in LLM applications.

Learn from Black Hat trainers and pen 

recursive character splitting(recommended)


In [43]:
recursive_splitter=RecursiveCharacterTextSplitter(chunk_size=80,chunk_overlap=20,length_function=len)
char_chunks=recursive_splitter.split_text(text)
print(f"created {len(char_chunks)} chunks")
print(f"First chunk: {char_chunks[0][:10]}...")
print("\n__________________")
print(char_chunks[0])
print("\n__________________")
print(char_chunks[1])
print("\n__________________")

created 18 chunks
First chunk: Python is ...

__________________
Python is an interpreted, high-level and general-purpose programming language.

__________________
language. Python's design philosophy emphasizes code readability with its

__________________


token based splitting

In [46]:
token_splitter=TokenTextSplitter(chunk_size=100,chunk_overlap=20)
token_chunks=token_splitter.split_text(text)
print(f"created {len(token_chunks)} chunks")
print(f"First chunk: {token_chunks[0][:10]}...")

created 3 chunks
First chunk: Python is ...
